In [1]:
import os
import string
import subprocess
import numpy as np
from collections import OrderedDict
import math
import pandas as pd
import geopandas as gpd
from scipy.spatial import Delaunay
from shapely.geometry import MultiLineString
from shapely.ops import cascaded_union, polygonize
import time


In [2]:
import pcraster as pcr


In [3]:
import mapIO


In [4]:
import pcrRecipes


In [5]:
import measures_py3 as msr


In [6]:
start_time = time.time()


In [7]:
output_dir = "/scratch/depfg/sutan101/tmp_menno/out/"
input_dir = "/scratch/depfg/hydrowld/river_scape/source/from_menno/riverscape/input/"
scratch_dir  = "/scratch/depfg/sutan101/tmp_menno/tmp/"

In [8]:
pcr.setglobaloption('unittrue')


In [9]:
pcrRecipes.make_dir(scratch_dir)
os.chdir(scratch_dir)


In [10]:
#-define input
settings = OrderedDict([
                    ('smoothing_percentage', 100),
                    ('smoothing_ecotope', 'UG-2'),
                    ('smoothing_trachytope', 1201),
                    
                    ('lowering_percentage', 100),
                    ('lowering_ecotope', 'UG-2'),
                    ('lowering_trachytope', 1201),
                    ('lowering_height', 'water_level_50d'),
                    
                    ('channel_width', 75),
                    ('channel_depth', 2.5),
                    ('channel_slope', 1./3.),
                    ('channel_ecotope', 'RnM'),
                    ('channel_trachytope', 105),
                    
                    ('relocation_alpha', 10000),
                    ('relocation_depth', 'AHN'),
                    ('relocation_ecotope', 'HG-2'),
                    ('relocation_trachytope', 1201),
                    
                    ('groyne_ref_level', 'wl_exc150d'),
                    ('minemb_ref_level', 'wl_exc50d'),
                    ('main_dike_dh', 0.50),
                    ])

In [11]:
current_dir = os.path.join(input_dir, 'reference_maps')
pcr.setclone(os.path.join(current_dir, 'clone.map'))
print(current_dir)

/scratch/depfg/hydrowld/river_scape/source/from_menno/riverscape/input/reference_maps


In [12]:
print(input_dir)
print(os.path.join(input_dir, 'reference_maps'))
print(current_dir)

/scratch/depfg/hydrowld/river_scape/source/from_menno/riverscape/input/
/scratch/depfg/hydrowld/river_scape/source/from_menno/riverscape/input/reference_maps
/scratch/depfg/hydrowld/river_scape/source/from_menno/riverscape/input/reference_maps


In [13]:
#- Import data
main_dike = msr.read_dike_maps(current_dir, 'main_dike')
minemb = msr.read_dike_maps(current_dir, 'minemb')
groynes = msr.read_dike_maps(current_dir, 'groyne')
hydro = msr.read_hydro_maps(current_dir)
mesh = msr.read_mesh_maps(current_dir)
axis = msr.read_axis_maps(current_dir)
lulc = msr.read_lulc_maps(current_dir)
geom = msr.read_geom_maps(current_dir)

In [14]:
pcr.aguila(os.path.join(current_dir, 'clone.map'))

In [15]:
pcr.aguila(geom.dem)

In [16]:
#- initiate the model
waal = msr.River('Waal', axis, main_dike, minemb, groynes, hydro, 
             mesh, lulc, geom)
waal_msr = msr.RiverMeasures(waal)
waal_msr.settings = settings


In [17]:
# Test measures
mask = pcr.boolean(1)
ID = 'everywhere'

In [18]:
lowering_msr = waal_msr.lowering_measure(settings, mask=mask, ID=ID)


In [19]:
lowering_msr.plot()

In [20]:

os.system("killall aguila")

0

In [21]:
groyne_low_msr = waal_msr.groyne_lowering_msr(settings, mask=mask, ID=ID)




In [22]:
minemb_low_msr = waal_msr.minemb_lowering_msr(settings, mask=mask, ID=ID)


In [23]:
main_dike_raise_msr = waal_msr.main_dike_raising_msr(settings, mask=mask, ID=ID)


In [24]:
large_sections = pcr.ifthen(pcr.areaarea(waal.geom.flpl_wide) > 1e6, waal.geom.flpl_wide)

flpl_section = pcr.ifthen(large_sections == 1, pcr.nominal(1))
pcr.aguila(flpl_section) 


In [25]:
test = list(np.unique(pcr.pcr2numpy(large_sections, -9999))[1:])
test[:]
for ID in test[:]:
    print(ID)

1
2
3
5
6
8
10
12
13
14
17
18
20
21
22
23
27
29
30
35
39
40
43
44
49
54
56


In [26]:
chan_msr = waal_msr.side_channel_measure(settings, mask=mask, ID=ID)


[1, 2, 3, 8, 10, 13, 14, 18, 20, 22, 23, 27, 30, 39, 40, 43, 49, 54, 56]
1
2
3
8
10
13
14
18
20
22
23
27
30
39
40
43
49
54
56


In [27]:
os.system("killall aguila")


256

In [28]:
smooth_msr = waal_msr.smoothing_measure(settings, mask=mask, ID=ID)

In [29]:
msr_list = [groyne_low_msr, minemb_low_msr,
            main_dike_raise_msr, lowering_msr, chan_msr, smooth_msr]


In [30]:
msr_root_dir = os.path.join(output_dir, 'measures_ensemble03/maps')

In [31]:
pcrRecipes.make_dir(msr_root_dir)

In [32]:
from measures_py3 import *

In [ ]:
for measure in msr_list:
    write_measure(measure, msr_root_dir)

/scratch/depfg/sutan101/tmp_menno/out/measures_ensemble03/maps/groynelowering_everywhere
/scratch/depfg/sutan101/tmp_menno/out/measures_ensemble03/maps/minemblowering_everywhere
